<h2>IBM Data Science Professional Certificate Capstone Project

<h4>Importing Libraries

In [48]:
import pandas as pd
import numpy as np

<h4>Integrate with Forsquare

In [49]:
# Foursquare Credentials
CLIENT_ID = 'ZE1Y4QGZYFGQTAKNNV1Y5JMDQ0R5CB4P2R2L4Y1GHFZNUMBJ' # your Foursquare ID
CLIENT_SECRET = 'R4WOIUKZ0X4PTOCHVKT3CDSDZBWRACXQNSQBDV2ZON2PBHUQ' # your Foursquare Secret
VERSION = '20200519' # Foursquare API version
LIMIT = 400 # Limit results to 400

<h4>Define custom functions

In [50]:
from math import sin,cos,radians,asin,sqrt # the sine, cosine, radians, arcsin, square root functions
import requests

def distance(coor1, coor2):   
    """This function calculates the distance (in kilometres) between two sets of latitude/longitude coordinates (coor1 and coor2), using the haversine formula (https://en.wikipedia.org/wiki/Haversine_formula)"""
    hav = lambda x: sin(x/2)**2
    coor1[0], coor1[1], coor2[0], coor2[1] = radians(coor1[0]), radians(coor1[1]), radians(coor2[0]), radians(coor2[1])
    return(2*6371*asin(sqrt(hav(coor2[0] - coor1[0]) + cos(coor1[0])*cos(coor2[0])*hav(coor2[1] - coor1[1]))))


def getNearbyVenues(names, latitudes, longitudes, query="restaurant", radius=3000):
    """This function returns the nearby venues at the specified location from the Foursquare API within 3km"""
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
            
        # make the GET request
        for a in range(0,5):
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']

                # return only relevant information for each nearby venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
                break
            except:
                print(str(name) + " - error in accessing venues. Making another attempt...")

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    nearby_venues.drop_duplicates(subset = ["Venue Latitude", "Venue Longitude", "Venue"], inplace = True)
    nearby_venues.reset_index(drop = True, inplace = True)
    
    return(nearby_venues)

<h4>Import LA city data

In [70]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_0be019a795d14b34a6d96d962284ac07 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1XayFCj99l64WisDL8Cmpq57kD62Fn52igqPfvNlPmje',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0be019a795d14b34a6d96d962284ac07.get_object(Bucket='appliedpythonfordatasciencecapsto-donotdelete-pr-uuy6yfhq43ptw7',Key='lacities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

postcodes = pd.read_csv(body)
postcodes.head()


,city,state_id,state_name,county_fips,county_name,county_name_all,lat,lng,population,density,ranking,zips,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173,Unnamed: 174,Unnamed: 175,Unnamed: 176,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186,Unnamed: 187,Unnamed: 188,Unnamed: 189,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199,Unnamed: 200,Unnamed: 201,Unnamed: 202,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206
0,El Dorado Hills,CA,California,6017,El Dorado,El Dorado,38.6750,-121.0490,45104,359.0,3,95672,95762.0,95682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lemon Cove,CA,California,6107,Tulare,Tulare,36.3790,-119.0312,232,107.0,3,93244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

<h4>Clean up the date. Each postcode should only be represented once.

In [72]:
postcodes = postcodes.groupby(["zips", "state_name", "lat", "lng"], sort = False, as_index = False).agg('|'.join)

In [53]:
postcodes = postcodes[['city','county_name','lat','lng','population','density','ranking','zips']]

In [73]:
postcodes.head

<bound method NDFrame.head of        zips  state_name      lat       lng                               city  \
0     95672  California  38.6750 -121.0490                    El Dorado Hills   
1     93244  California  36.3790 -119.0312                         Lemon Cove   
2     94929  California  38.2436 -122.9560                       Dillon Beach   
3     93291  California  36.3795 -119.2956                    Patterson Tract   
4     95569  California  40.3987 -123.9474                           Redcrest   
5     93637  California  36.9640 -120.0803                             Madera   
6     95490  California  39.4429 -123.3963                        Brooktrails   
7     95441  California  38.7173 -122.9034                        Geyserville   
8     95315  California  37.4306 -120.7759                              Delhi   
9     90680  California  33.8002 -117.9935                            Stanton   
10    95426  California  38.8356 -122.7230                               Cobb  

<h4>Drop all of the postcodes that are not in LA county.LA postcodes range from 90001 to 90559.

In [86]:
postcodes.drop(postcodes[postcodes["zips"]<90001].index, inplace = True)
postcodes.drop(postcodes[postcodes["zips"]>90559].index, inplace = True)
postcodes.reset_index(drop=True, inplace = True)

# Obtain latitude and longitude for the city center (postcode 90001)
CC_lat = postcodes[postcodes["zips"]==90001]["lat"].values[0]
CC_lon = postcodes[postcodes["zips"]==90001]["lng"].values[0]

# Calculate the distance from the city centre for each postcode
DistanceKM = []
for a in range(0,postcodes.shape[0]):
    lat = postcodes.loc[a, "lat"]
    lon = postcodes.loc[a, "lng"]
    DistanceKM.append(distance([CC_lat, CC_lon], [lat,lon]))
postcodes["FromCity_KM"] = DistanceKM

# Drop all of the postcodes that are more than 60 kilometres from the city centre
postcodes.drop(postcodes[postcodes["FromCity_KM"]>60].index, inplace = True)
postcodes.reset_index(drop=True, inplace = True)

postcodes.shape

(46, 12)

<h4>Download list of restaurants from the Foursquare API

In [87]:
la_venues = getNearbyVenues(names=postcodes['zips'],
                                   latitudes=postcodes['lat'],
                                   longitudes=postcodes['lng'],
                                   query="Restaurant"
                                  )

la_venues.shape

90305
90266
90059
90221
90058
90220
90274
90008
90274
90277
90255
90304
90248
90001
90245
90250
90254
90242
90212
90040
90290
90250
90262
90255
90201
90401
90260
90044
90069
90201
90248
90232
90201
90274
90044
90063
90280
90292
90265
90278
90291
90008
90249
90270
90275
90247


(2526, 7)

<h4>Get Japanese restaurants

In [88]:
japanese_venues = getNearbyVenues(names=postcodes['zips'],
                                   latitudes=postcodes['lat'],
                                   longitudes=postcodes['lng'],
                                   query="Japanese restaurant"
                                  )

japanese_venues

90305
90266
90059
90221
90058
90220
90274
90008
90274
90277
90255
90304
90248
90001
90245
90250
90254
90242
90212
90040
90290
90250
90262
90255
90201
90401
90260
90044
90069
90201
90248
90232
90201
90274
90044
90063
90280
90292
90265
90278
90291
90008
90249
90270
90275
90247


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,90305,33.9566,-118.3444,Yoshinoya,33.944800,-118.352970,Donburi Restaurant
1,90266,33.8895,-118.3972,The Izaka-ya by Katsuya,33.885322,-118.409885,Japanese Restaurant
2,90266,33.8895,-118.3972,O Sho,33.885083,-118.396479,Japanese Restaurant
3,90266,33.8895,-118.3972,Fusion Sushi,33.885793,-118.408855,Japanese Restaurant
4,90266,33.8895,-118.3972,Umi by Hamasaku,33.902491,-118.394463,Japanese Restaurant
5,90266,33.8895,-118.3972,Akatora,33.900956,-118.417719,Japanese Restaurant
6,90266,33.8895,-118.3972,Pisces Sushi,33.898311,-118.416640,Sushi Restaurant
7,90266,33.8895,-118.3972,SanSai Japanese Grill,33.901385,-118.373559,Japanese Restaurant
8,90266,33.8895,-118.3972,Fusion Sushi,33.864155,-118.392755,Sushi Restaurant
9,90266,33.8895,-118.3972,Gu Gu Sushi & Roll,33.865773,-118.387546,Sushi Restaurant


<h4>For each city, calculate the distance to the nearest Japanese restaurant

In [89]:
def distance_from_closest(lat, long, venues):
    """From a nominated latitude (lat) and longitude (lon), this function will return the distance of the closest point from a set of coordinates (venues)"""
    dist = []
    for a in range(0, venues.shape[0]):
        dist.append(distance([lat,long], [venues.loc[a, "Venue Latitude"], venues.loc[a, "Venue Longitude"]]))
    return(np.min(dist))

DistToNearestJapanese = []

for b in range(0, postcodes.shape[0]):
    DistToNearestJapanese.append(distance_from_closest(postcodes.loc[b, "lat"],
                                                      postcodes.loc[b, "lng"],
                                                      japanese_venues))

postcodes["FromNearestJapanese_KM"] = DistToNearestJapanese
postcodes

,zips,state_name,lat,lng,city,county_name,county_name_all,state_id,FromCity_KM,FromNearestJapanese_KM,NumVenues,Diversity
0,90305,California,33.9566,-118.3444,Inglewood,Los Angeles,Los Angeles,CA,9.284918,1.531840,116,0.940547
1,90266,California,33.8895,-118.3972,Manhattan Beach,Los Angeles,Los Angeles,CA,16.569366,0.215189,132,0.953857
2,90059,California,33.9199,-118.2362,Willowbrook,Los Angeles,Los Angeles,CA,5.427649,2.234065,96,0.906684
3,90221,California,33.8949,-118.1956,East Rancho Dominguez,Los Angeles,Los Angeles,CA,9.324818,1.340359,90,0.911852
4,90058,California,34.0019,-118.2106,Vernon,Los Angeles,Los Angeles,CA,4.891533,1.170341,98,0.898792
5,90220,California,33.8930,-118.2275,Compton,Los Angeles,Los Angeles,CA,8.511101,0.521179,87,0.913199
6,90274,California,33.7600,-118.3471,Rolling Hills,Los Angeles,Los Angeles,CA,25.007033,2.157803,26,0.911243
7,90008,California,33.9972,-118.3740,Ladera Heights,Los Angeles,Los Angeles,CA,12.350323,1.526436,120,0.955833
8,90274,California,33.7783,-118.3508,Rolling Hills Estates,Los Angeles,Los Angeles,CA,23.277265,1.572729,101,0.958730
9,90277,California,33.8346,-118.3417,Torrance,Los Angeles,Los Angeles,CA,17.344552,0.352353,100,0.956800


<h4>For each cities, calculate the total number of restaurants within a 3km radius.Note:there is some overlap; a venue may be counted more than once for adjacent suburbs

In [90]:
n_venues = []
for b in range(0, postcodes.shape[0]):
    lat1 = postcodes.loc[b, "lat"]
    lon1 = postcodes.loc[b, "lng"]    
    dist = []
    for c in range(0, la_venues.shape[0]):
        lat2 = la_venues.loc[c, "Venue Latitude"]
        lon2 = la_venues.loc[c, "Venue Longitude"]
        dist.append(distance([lat1, lon1], [lat2, lon2]))
    la_venues["FromPostcode_KM"] = dist
    n_venues.append(la_venues[la_venues["FromPostcode_KM"]<=3].shape[0])

postcodes["NumVenues"] = n_venues
postcodes.head()

,zips,state_name,lat,lng,city,county_name,county_name_all,state_id,FromCity_KM,FromNearestJapanese_KM,NumVenues,Diversity
0,90305,California,33.9566,-118.3444,Inglewood,Los Angeles,Los Angeles,CA,9.284918,1.531840,116,0.940547
1,90266,California,33.8895,-118.3972,Manhattan Beach,Los Angeles,Los Angeles,CA,16.569366,0.215189,132,0.953857
2,90059,California,33.9199,-118.2362,Willowbrook,Los Angeles,Los Angeles,CA,5.427649,2.234065,96,0.906684
3,90221,California,33.8949,-118.1956,East Rancho Dominguez,Los Angeles,Los Angeles,CA,9.324818,1.340359,90,0.911852
4,90058,California,34.0019,-118.2106,Vernon,Los Angeles,Los Angeles,CA,4.891533,1.170341,98,0.898792


<h4>Calculate the diversity of each neighbourhood, in terms of the variety of restaurant type within 3km of its coordinates

In [91]:
def gini_simpson(cat_list):
    """Calculates the Gini-Simpson diversity index from a list of categories (cat_list) provided.
    See: https://en.wikipedia.org/wiki/Diversity_index#Gini%E2%80%93Simpson_index
    Higher values indicate greater diversity."""
    unq_list = np.unique(np.array(cat_list), return_counts = True)
    p = unq_list[1] / np.sum(unq_list[1])
    return(1 - np.sum(p**2))

diversity = []
for b in range(0, postcodes.shape[0]):
    lat1 = postcodes.loc[b, "lat"]
    lon1 = postcodes.loc[b, "lng"]    
    dist = []
    for c in range(0, la_venues.shape[0]):
        lat2 = la_venues.loc[c, "Venue Latitude"]
        lon2 = la_venues.loc[c, "Venue Longitude"]
        dist.append(distance([lat1, lon1], [lat2, lon2]))
    la_venues["FromPostcode_KM"] = dist
    diversity.append(gini_simpson(la_venues.loc[la_venues["FromPostcode_KM"]<=3, "Venue Category"].values))

postcodes["Diversity"] = diversity

# If there are fewer than 10 venues, don't calculate a diversity score
postcodes.loc[postcodes["NumVenues"]<10, "Diversity"] = np.nan

In [92]:
postcodes.head()

,zips,state_name,lat,lng,city,county_name,county_name_all,state_id,FromCity_KM,FromNearestJapanese_KM,NumVenues,Diversity
0,90305,California,33.9566,-118.3444,Inglewood,Los Angeles,Los Angeles,CA,9.284918,1.531840,116,0.940547
1,90266,California,33.8895,-118.3972,Manhattan Beach,Los Angeles,Los Angeles,CA,16.569366,0.215189,132,0.953857
2,90059,California,33.9199,-118.2362,Willowbrook,Los Angeles,Los Angeles,CA,5.427649,2.234065,96,0.906684
3,90221,California,33.8949,-118.1956,East Rancho Dominguez,Los Angeles,Los Angeles,CA,9.324818,1.340359,90,0.911852
4,90058,California,34.0019,-118.2106,Vernon,Los Angeles,Los Angeles,CA,4.891533,1.170341,98,0.898792


<h4>Drop cities that are more than 30 kilometres from the city. And rank them.

In [93]:
postcodes_filtered = postcodes.copy()
postcodes_filtered.drop(postcodes_filtered[postcodes_filtered["FromCity_KM"]>30].index, inplace = True)
postcodes_filtered.reset_index(drop=True, inplace = True)

In [94]:
postcodes_filtered["FromNearestJapanese_KM"] = postcodes_filtered["FromNearestJapanese_KM"].rank(ascending = False)
postcodes_filtered["NumVenues_rank"] = postcodes_filtered["NumVenues"].rank(ascending = False)
postcodes_filtered["Diversity_rank"] = postcodes_filtered["Diversity"].rank(ascending = False)

overall = []
for a in range(0,postcodes_filtered.shape[0]):  
    overall.append(np.sum([postcodes_filtered.loc[a, "FromNearestJapanese_KM"],
                           postcodes_filtered.loc[a, "NumVenues_rank"],
                           postcodes_filtered.loc[a, "Diversity_rank"]]))

postcodes_filtered["Overall"] = overall
postcodes_filtered.sort_values(by=["Overall"])[0:21]

,zips,state_name,lat,lng,city,county_name,county_name_all,state_id,FromCity_KM,FromNearestJapanese_KM,NumVenues,Diversity,NumVenues_rank,Diversity_rank,Overall
30,90232,California,34.0058,-118.3968,Culver City,Los Angeles,Los Angeles,CA,14.633429,32.0,158,0.963708,2.0,1.0,35.0
7,90008,California,33.9972,-118.3740,Ladera Heights,Los Angeles,Los Angeles,CA,12.350323,14.0,120,0.955833,16.5,8.0,38.5
40,90249,California,33.8885,-118.3350,Alondra Park,Los Angeles,Los Angeles,CA,12.163460,25.0,151,0.953292,4.0,11.0,40.0
37,90278,California,33.8574,-118.3766,Redondo Beach,Los Angeles,Los Angeles,CA,17.318801,35.0,148,0.959551,5.0,2.0,42.0
31,90201,California,33.9631,-118.1830,Cudahy,Los Angeles,Los Angeles,CA,5.718289,9.0,146,0.919216,6.0,30.0,45.0
8,90274,California,33.7783,-118.3508,Rolling Hills Estates,Los Angeles,Los Angeles,CA,23.277265,12.0,101,0.958730,31.0,3.0,46.0
15,90250,California,33.9168,-118.3693,Del Aire,Los Angeles,Los Angeles,CA,12.836582,23.0,138,0.949590,8.0,15.0,46.0
35,90280,California,33.9448,-118.1926,South Gate,Los Angeles,Los Angeles,CA,5.464519,15.0,153,0.920672,3.0,29.0,47.0
16,90254,California,33.8654,-118.3966,Hermosa Beach,Los Angeles,Los Angeles,CA,18.086682,36.0,135,0.958025,9.0,4.0,49.0
11,90304,California,33.9380,-118.3586,Lennox,Los Angeles,Los Angeles,CA,11.029288,24.0,129,0.949703,11.0,14.0,49.0


<h4>map them of top 20 recommended location

In [96]:
import folium

latitude = la_venues[la_venues["Neighborhood"]==90001]["Neighborhood Latitude"].values[0]
longitude = la_venues[la_venues["Neighborhood"]==90001]["Neighborhood Longitude"].values[0]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
#markers_colors = []

for lat, lon, suburb in zip(postcodes_filtered.sort_values(by=["Overall"])[0:21]["lat"].values,
                            postcodes_filtered.sort_values(by=["Overall"])[0:21]["lng"].values,
                            postcodes_filtered.sort_values(by=["Overall"])[0:21]["city"].values):
    label = folium.Popup(suburb, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)

map_clusters